In [1]:
from tensorflow.keras.layers import (Input, Conv2D, Flatten, MaxPooling2D,
                                     Dense, Dropout, Concatenate)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import numpy as np
import os
import pickle

In [2]:
# # Uncomment this box if the file is in google colab
# path2root = "/content/drive/MyDrive/Projects/EyeTracker/"

In [2]:
# Uncomment this box if the file is in your local directory
path2root = "../../"

In [3]:
subjects_fol = "subjects/"
data_boi_fol = "data-boi/"
models_fol = "models/"
models_boi_fol = "boi/"
raw_fol = "raw/"
# chosen_inputs = [0, 1, 2, 6, 7, 8, 9]
chosen_inputs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
data_boi_dir = path2root + subjects_fol + f"{3}/" + data_boi_fol

with open(data_boi_dir + "x1.pickle", "rb") as f:
    x1 = pickle.load(f)
with open(data_boi_dir + "x2.pickle", "rb") as f:
    x2 = pickle.load(f)
with open(data_boi_dir + "y.pickle", "rb") as f:
    y = pickle.load(f)

x2_chs_inp = x2[:, chosen_inputs]

In [10]:
input1 = Input(x1.shape[1:])
layer = Conv2D(16, (11, 11), (1, 1), "same", activation="relu")(input1)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Conv2D(32, (7, 7), (1, 1), "same", activation="relu")(layer)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Conv2D(64, (5, 5), (1, 1), "same", activation="relu")(layer)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Conv2D(128, (3, 3), (1, 1), "same", activation="relu")(layer)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Flatten()(layer)

layer = Dense(256, "relu")(layer)

input2 = Input(x2_chs_inp.shape[1:])
layer = Concatenate()([layer, input2])

layer = Dense(128, "relu")(layer)

# layer = Dense(64, "relu")(layer)

layer = Dense(32, "relu")(layer)

layer = Dense(8, "relu")(layer)

output_layer = Dense(1, "sigmoid")(layer)

input_layers = [input1, input2]

model = Model(inputs=input_layers, outputs=output_layer)

model.compile(optimizer="adam", loss="binary_crossentropy")

print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 100, 100, 16) 1952        input_5[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 50, 50, 16)   0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 50, 50, 32)   25120       max_pooling2d_8[0][0]            
____________________________________________________________________________________________

In [7]:
# plot_model(model, show_shapes=True)

In [6]:
models_dir = path2root + models_fol
if not os.path.exists(models_dir):
    os.mkdir(models_dir)

models_boi_dir = models_dir + models_boi_fol
if not os.path.exists(models_boi_dir):
    os.mkdir(models_boi_dir)

raw_dir = models_boi_dir + raw_fol
if not os.path.exists(raw_dir):
    os.mkdir(raw_dir)

In [11]:
models_numbers = []
models_name = os.listdir(raw_dir)
if models_name:
    for model_name in models_name:
        model_num = int(model_name[5:-3])
        models_numbers.append(model_num)
    max_num = max(models_numbers)
else:
    max_num = 0

model.save(raw_dir + f"model{max_num+1}.h5")